# Satellite Orbit Calculation for Observation
Written by Kiyoaki Okudaira<br>
*Kyushu University Hanada Lab / University of Washington / IAU CPS SatHub<br>
(okudaira.kiyoaki.528@s.kyushu-u.ac.jp or kiyoaki@uw.edu)<br>
<br>
Calculate satellite orbit by SGP4 propagation for ground-based observation and generate sequence files for telescope control softwares<br>
<br>
**History**<br>
coding 2025-12-07 : 1st coding<br>
update 2025-12-08 : Sequence generator for BULL's eye Sequencer (.pws) and SharpCap 4.1 (.scs)<br>
update 2025-12-17 : Fixed epoch option for TEME to J 2000<br>
update 2026-02-13 : Add heavens-above.com PassSkyChart<br>
<br>
(c) 2026 Kiyoaki Okudaira - Kyushu University Hanada Lab (SSDL) / University of Washington / IAU CPS SatHub

### Parameters
**Observation date**<br>
In UTC

In [ ]:
obs_begin = "2026-02-14T09:00:00.000" # UTC Date and time observation begin [YYYY-MM-DDTHH:MM:SS.SSS] | str
obs_end   = "2026-02-14T11:00:00.000" # UTC Date and time observation end   [YYYY-MM-DDTHH:MM:SS.SSS] | str
obs_step  = 1.000                     # Time step [sec] | float or int

**TLE Download settings**<br>
Downloading from space-track.org is highly recommended

In [ ]:
norad_id = 10365        # NORAD CATALOG ID | int
download_tle = True     # Download Latest TLE from clestrak or select preferred TLE file | bool
tle_source  = "space-track.org" # "space-track.org" or "celestrak.org" | str

**Heavens-Above settings**

In [ ]:
ha_passskychart = True # Download Pass Chart from heavens-above.com | bool
ha_timezone = "UCT"    # Pass Chart display timezone : "UCT" or "JapST" (etc) | str
ha_imgsize  = 1600     # Pass Chart image size [pix] ; Recommend 800 | int

**Sequence File Settings**<br>
Sequence file for telescope control software (BULL's eye Sequencer and SharpCap 4.1)

In [ ]:
lst_h = +9          # Local Time to UTC [hour] | int
lst_m = +00         # Local Time to UTC [min]  | int
obs_type = "LEO"    # "GEO" or "LEO" | str

**Capture Settings**<br>
Capture settings for SharpCap 4.1

In [ ]:
from input.sensor.QHY294PROM import * # Sensor setting file
obs_ftype      = "FITS"   # Image output filetype | str
ccd_colorspace = "MONO16" # CCD color mode | str
ccd_capmode    = "DUR"    # Capture mode "DUR" / "FRM" | str

ccd_offset     = 8        # CCD offset | int
ccd_binning    = 1        # CCD binning | int
ccd_gain       = 2600     # CCD gain | int
ccd_exp        = 1/1000   # Exposure [sec] | int or float

ccd_frame      = 12       # Frame counts when ccd_capmode "FRM" [sec] | int

ccd_auto_dur   = True     # If True, capture duration is determined automatically | bool
ccd_dur        = 120      # Capture duration when ccd_capmode "DUR" [sec] | int or float

obsdate_offset = 0        # Observation begin offset [sec] | int or float

obs_timing     = ""     # If "HIGH", capture starts at highest elevation | str
cap_el_min     = 20     # Capture elevation min [deg] | int or float

**Advanced**

In [ ]:
of_epoch = False # Recommend False | bool (True/False/None)
j2000_utc = "2025-12-22T09:28:50" # If of_epoch is None : Fixed epoch for J2000 conversion [mjd] | str

### Import and initial settings

**PATH settings**

In [ ]:
base_PATH = "/Users/kiyoaki/VScode/satphotometry_package/"
output_PATH = base_PATH + "output"
input_PATH = base_PATH + "input"
spice_myfile_PATH = "/Users/kiyoaki/VScode/satphotometry_package/config/myfile.txt"
tle_PATH = "/Users/kiyoaki/VScode/satphotometry_package/input/tle/10439.tle" # If download_tle is False | str

**SPICE toolkit**<br>
import spiceypy and start up SPICE kernel

In [ ]:
import spiceypy as spice
spice.furnsh(spice_myfile_PATH)

**Standard libraries**

In [ ]:
from datetime import datetime, timedelta
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import path
import json

import astropy.units as u
from astropy.coordinates import Angle
from astropy.table import Table
from astropy.time import Time

**Satphotometry library**<br>
satphotometry.satorbit must be imported after starting up SPICE kernel

In [ ]:
from satphotometry import satorbit,gettle,heavens_above

**Observatory setting**

In [ ]:
from input.obs_site.KUPT import *
# obs_gd_lon_deg = 130.0 + (12.0 + 42.0 / 60.0) / 60.0    # [deg]
# obs_gd_lat_deg = 33.0 + (35.0 + 56.0 / 60.0) / 60.0     # [deg]
# obs_gd_height  = 0.073  # [km]

# wavelength_m = 0.5e-6   #[m]
# aperture_m   = 508.0e-3 #[m]

obs_gd_lon = math.radians(obs_gd_lon_deg) # [radian]
obs_gd_lat = math.radians(obs_gd_lat_deg) # [radian]

**Space-track.org settings**<br>
Set your space-track.org identity and password at `./config/space-track-org.config`

In [ ]:
if tle_source == "space-track.org":
    with open(f'{base_PATH}config/space-track-org.config') as f:
        st_config = json.load(f)
        st_user_id  = st_config["identity"] # space-track.org user id | str
        st_password = st_config["password"] # space-track.org password | str

**Planetary constants**<br>
Get the Earth's constants that are needed for SGP4 propagation<br>
NAIF integer ID code of the Earth is 399

In [ ]:
earth_constants = satorbit.get_planetconst(399,["J2", "J3", "J4", "KE", "QO", "SO", "ER", "AE"])

### Orbit calculation

**Observation date and time**

In [ ]:
obs_begin_et = spice.utc2et(obs_begin)
obs_end_et   = spice.utc2et(obs_end)
obs_ets      = [obs_begin_et+i*obs_step for i in range(0,int((obs_end_et-obs_begin_et)//obs_step+1))]

**Output list**

In [ ]:
output = []

**Download TLE file**<br>
Download latest Two-Line Element set from space-track.org or celestrak.org

In [ ]:
if download_tle is True:
    status = 0
    if tle_source == "space-track.org":
        status,tle_result = gettle.space_track.get_latest_TLE(norad_id,st_user_id,st_password)
    if tle_source != "space-track.org" or status != 200:
        status,tle_result = gettle.celes_trak.get_latest_TLE(norad_id)

    if status == 200:
        tle_fname = "{0}_{1}".format(norad_id,gettle.parse_tle2epoch_fname(tle_result.splitlines()[1]))
        tle_PATH = "{0}/tle/{1}.tle".format(input_PATH,tle_fname)
        with open(tle_PATH,mode="w") as f:
            f.write(tle_result)
    else:
        raise ValueError("Failed to download Two-Line Element set or NORAD catalog number is invalid.")

**Read TLE file**

In [ ]:
satname,line1,line2 = satorbit.read_TLEfile(tle_PATH)
epoch,elems = satorbit.parse_TLE2element(line1,line2)

epoch_utc = spice.et2utc(epoch, "ISOC", 0)

if satname is not None:
    if satname[0:2] == "0 ":
        objname = satname[2:]
    objname = objname.rstrip()
else:
    objname = None
norad_id = int(line1[2:7])
intldes = line1.split(" ")[2]
if int(intldes[:2]) >= 57:
    intldes = "19" + intldes[:2] + "-" + intldes[2:]
else:
    intldes = "20" + intldes[:2] + "-" + intldes[2:]

**SGP4 Propagation**

In [ ]:
for et in obs_ets:
    et_utc = spice.et2utc(et, "ISOC", 0)

    state_teme = spice.evsgp4(et, earth_constants, elems)

    obs_itrf = satorbit.geo2itrf(obs_gd_lon,obs_gd_lat,obs_gd_height)
    obs_j2000 = satorbit.itrf2J2000(obs_itrf,et)

    if of_epoch:
        state_j2000 = satorbit.teme2J2000(state_teme,epoch)
    elif of_epoch is None:
        et_j2000 = spice.utc2et(j2000_utc)
        state_j2000 = satorbit.teme2J2000(state_teme,et_j2000)
    else:
        state_j2000 = satorbit.teme2J2000(state_teme,et)

    range_km,ra,dec = satorbit.J20002radec(state_j2000[0:3],obs_j2000)

    ra_hms = Angle(ra*u.rad).to_string(unit=u.hourangle, sep=':', precision=1)
    ra_hms = "+0"+ra_hms if len(ra_hms)<10 else "+"+ra_hms

    dec_dms = Angle(dec*u.rad).to_string(unit=u.deg, sep=':', precision=1)
    dec_dms = "+0"+dec_dms if (dec>0 and len(dec_dms)<10) else ("+"+dec_dms if dec>0 else ("-0"+dec_dms[1:] if len(dec_dms)<11 else dec_dms))
    
    ra_deg = math.degrees(ra)
    dec_deg = math.degrees(dec)

    state_itrf = satorbit.J20002itrf(state_j2000,et)
    _,az,el = satorbit.itrf2azel(state_itrf[0:3],obs_itrf,obs_gd_lon,obs_gd_lat)

    az_deg = math.degrees(az)
    el_deg = math.degrees(el)

    in_umbra = satorbit.check_umbra(state_j2000[0:3],et)
    phase = satorbit.phase_angle(state_j2000[0:3],obs_j2000,et)
    phase_deg = math.degrees(phase)

    apparent_v_km_s = satorbit.apparent_v(state_j2000,obs_itrf,et)
    res_km = range_km * (wavelength_m / aperture_m)
    ex_ms = res_km / apparent_v_km_s

    output.append(
        [
            et_utc,
            in_umbra,
            round(range_km,4),
            ra_hms,
            dec_dms,
            round(ra_deg,4),
            round(dec_deg,4),
            round(phase_deg,4),
            round(az_deg,4),
            round(el_deg,4),
            round(apparent_v_km_s,4),
            round(res_km*1000,4),
            round(1/ex_ms,4)
            ]
        )

**SPICE toolkit kernel clear**

In [ ]:
spice.kclear()

### Output
**Save output file**

In [ ]:
output = Table(
    list(zip(*output)),
    names=(
        "YYYY-MM-DDThh:mm:ss",
        "umbra",
        "range[km]",
        "ra[hh:mm:ss.s]",
        "dec[dd:mm:ss.s]",
        "ra[deg]",
        "dec[deg]",
        "pha[deg]",
        "az[deg]",
        "el[deg]",
        "v[km/s]",
        "res[m]",
        "ex[1/s]"
        )
    )
output.write("{0}/orbit/{1}_{2}_output.csv".format(output_PATH,norad_id,obs_begin[0:10]),overwrite=True)

**Display output**

In [ ]:
print("--------------------------------------------------------------------------------")
print(" Two-Line Element Set")
print("--------------------------------------------------------------------------------")
if satname is not None:
    print(" "+satname)
print(" "+line1)
print(" "+line2)
print("--------------------------------------------------------------------------------")
print(" Orbital Elements")
print("--------------------------------------------------------------------------------")
print(" INCL  [deg]     : {0}".format(round(math.degrees(elems[3]),4)))
print(" NODE0 [deg]     : {0}".format(round(math.degrees(elems[4]),4)))
print(" ECC   [ - ]     : {0}".format(round(elems[5],4)))
print(" OMEGA [deg]     : {0}".format(round(math.degrees(elems[6]),4)))
print(" M0    [deg]     : {0}".format(round(math.degrees(elems[7]),4)))
print(" N0    [deg/min] : {0}".format(round(math.degrees(elems[8]),4)))
print(" EPOCH [UTC]     : {0}".format(epoch_utc) )
print("--------------------------------------------------------------------------------")
print()
print("--------------------------------------------------------------------------------")
print(" Result")
print("--------------------------------------------------------------------------------")
output.pprint(max_width=-1, max_lines=-1)

**Plot**

In [ ]:
output_filtered = output[(output["umbra"] == False)]

# az-el
plt.figure(figsize=(8,4))
plt.plot(output_filtered["az[deg]"], output_filtered["el[deg]"], color='black')
plt.scatter(output_filtered["az[deg]"][0], output_filtered["el[deg]"][0],marker='.',color='blue',label=output_filtered["YYYY-MM-DDThh:mm:ss"][0])
plt.scatter(output_filtered["az[deg]"][-1], output_filtered["el[deg]"][-1],marker='.',color='red',label=output_filtered["YYYY-MM-DDThh:mm:ss"][-1])
plt.annotate(" {0} {1}".format(objname,intldes),(output_filtered["az[deg]"][0], output_filtered["el[deg]"][0]))
plt.annotate('', xy=(output_filtered["az[deg]"][-1], output_filtered["el[deg]"][-1]), xytext=(output_filtered["az[deg]"][-2], output_filtered["el[deg]"][-2]),
            arrowprops=dict(shrink=0, width=1, headwidth=4, 
                            headlength=6, connectionstyle='arc3',
                            facecolor='black', edgecolor='black')
            )
plt.xlabel('Azimuth [deg]')
plt.ylabel('Elevation [deg]')
plt.xlim(0,360)
plt.ylim(0,90)
plt.xticks(np.arange(0,390,30))
plt.yticks(np.arange(0,100,10))
plt.grid()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)
plt.savefig("{0}/orbit/{1}_{2}_az-al.png".format(output_PATH,norad_id,obs_begin[0:10]),dpi=640)
plt.savefig("{0}/orbit/{1}_{2}_az-al.pdf".format(output_PATH,norad_id,obs_begin[0:10]))
plt.show()

# pha-el
plt.figure(figsize=(8,4))
plt.plot(output_filtered["pha[deg]"], output_filtered["el[deg]"], color='black')
plt.scatter(output_filtered["pha[deg]"][0], output_filtered["el[deg]"][0],marker='.',color='blue',label=output_filtered["YYYY-MM-DDThh:mm:ss"][0])
plt.scatter(output_filtered["pha[deg]"][-1], output_filtered["el[deg]"][-1],marker='.',color='red',label=output_filtered["YYYY-MM-DDThh:mm:ss"][-1])
plt.annotate(" {0} {1}".format(objname,intldes),(output_filtered["pha[deg]"][0], output_filtered["el[deg]"][0]))
plt.annotate('', xy=(output_filtered["pha[deg]"][-1], output_filtered["el[deg]"][-1]), xytext=(output_filtered["pha[deg]"][-2], output_filtered["el[deg]"][-2]),
            arrowprops=dict(shrink=0, width=1, headwidth=4, 
                            headlength=6, connectionstyle='arc3',
                            facecolor='black', edgecolor='black')
            )
plt.xlabel('Solar Phase Angle [deg]')
plt.ylabel('Elevation [deg]')
plt.xlim(0,180)
plt.ylim(0,90)
plt.xticks(np.arange(0,210,30))
plt.yticks(np.arange(0,100,10))
plt.grid()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)
plt.savefig("{0}/orbit/{1}_{2}_pha-al.png".format(output_PATH,norad_id,obs_begin[0:10]),dpi=640)
plt.savefig("{0}/orbit/{1}_{2}_pha-al.pdf".format(output_PATH,norad_id,obs_begin[0:10]))
plt.show()

### Heavens-Above
**Get pass sky chart**<br>
Get pass sky chart from www.heavens-above.com/PassSkyChart2.ashx

In [ ]:
try:
    if ha_passskychart:
        query_result = heavens_above.get_pass_summary(norad_id,obs_gd_lon_deg,obs_gd_lat_deg,obs_gd_height,ha_timezone)
        obs_begin_mjd = Time(obs_begin, format="isot", scale="utc").mjd
        mjds = heavens_above.parse_summary2mjd(query_result)
        ha_mjd = mjds[(mjds-obs_begin_mjd).argmin()]
        ha_mjd = ha_mjd if abs(ha_mjd - obs_begin_mjd) < 1/48 else None

        query_result = heavens_above.get_pass_detail(norad_id,obs_gd_lon_deg,obs_gd_lat_deg,obs_gd_height,ha_mjd,ha_timezone)
        pass_id = heavens_above.parse_detail2passid(query_result)

        query_result = heavens_above.get_pass_chart(pass_id,obs_gd_lon_deg,obs_gd_lat_deg,obs_gd_height,ha_timezone,ha_imgsize)
        with open(input_PATH+"/heavens-above/PassSkyChart2.ashx.png", mode='wb') as f:
            f.write(query_result)
except:
    obs_begin_mjd = Time(obs_begin, format="isot", scale="utc").mjd
    query_result = heavens_above.get_wholeskychart(obs_gd_lon_deg,obs_gd_lat_deg,obs_gd_height,obs_begin_mjd,ha_timezone,ha_imgsize)
    with open(input_PATH+"/heavens-above/PassSkyChart2.ashx.png", mode='wb') as f:
        f.write(query_result)

**Display pass sky chart**

In [ ]:
if ha_passskychart:
    ha_data = mpimg.imread(input_PATH+"/heavens-above/PassSkyChart2.ashx.png")
    ha_radius = ha_imgsize/2
    ha_x = [ha_radius - ((90-output_filtered["el[deg]"][i])*ha_radius/90)*math.sin(math.radians(output_filtered["az[deg]"][i])) for i in range(len(output_filtered["el[deg]"]))]
    ha_y = [ha_radius - ((90-output_filtered["el[deg]"][i])*ha_radius/90)*math.cos(math.radians(output_filtered["az[deg]"][i])) for i in range(len(output_filtered["el[deg]"]))]
    ha_point_x = []
    ha_point_y = []
    for i in range(len(output_filtered["el[deg]"])):
        if output_filtered["YYYY-MM-DDThh:mm:ss"][i][-2:] == "00":
            ha_point_x.append(ha_radius - ((90-output_filtered["el[deg]"][i])*ha_radius/90)*math.sin(math.radians(output_filtered["az[deg]"][i])))
            ha_point_y.append(ha_radius - ((90-output_filtered["el[deg]"][i])*ha_radius/90)*math.cos(math.radians(output_filtered["az[deg]"][i])))

    plt.figure(figsize=(ha_imgsize/100,ha_imgsize/100))
    plt.imshow(ha_data)
    plt.plot(ha_x, ha_y, color='blue', linewidth=1)
    plt.scatter(ha_point_x, ha_point_y, color='blue', s=4)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['bottom'].set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    plt.tick_params(bottom=False, left=False, right=False, top=False, labelbottom=False, labelleft=False, labelright=False, labeltop=False)
    plt.savefig("{0}/orbit/{1}_{2}_heavens-above.png".format(output_PATH,norad_id,obs_begin[0:10]),dpi=800)
    plt.savefig("{0}/orbit/{1}_{2}_heavens-above.pdf".format(output_PATH,norad_id,obs_begin[0:10]))
    plt.show()
else:
    print("PassSkyChart not available")

### Sequence File

In [ ]:
def utc2lst(dt_utc,lst_h,lst_m,format="%Y-%m-%dT%H:%M:%S"):
    dt_utc = datetime.strptime(dt_utc, "%Y-%m-%dT%H:%M:%S")
    dt_jst = dt_utc + timedelta(hours=lst_h,minutes=lst_m)
    dt_jst = dt_jst.strftime(format)
    return dt_jst

**BULL's eye Sequencer (PWI4)**<br>
Sequence file (pws file)

In [ ]:
output_filtered = output_filtered[(output_filtered["el[deg]"]>=obs_el_min)]
obsdate_offset = obsdate_offset/60

if len(output_filtered) > 0:
    row_elmax = output_filtered[output_filtered["el[deg]"].argmax()]
    seq_begin = output_filtered[0]["YYYY-MM-DDThh:mm:ss"]
    seq_end   = output_filtered[-1]["YYYY-MM-DDThh:mm:ss"]

    seq_begin_lst = utc2lst(seq_begin,lst_h,lst_m)
    seq_end_lst   = utc2lst(seq_end,lst_h,lst_m)
        
    f = open("{0}/pws/{1}_{2}.pws".format(output_PATH,norad_id,obs_begin[0:10]),mode="w")

    print("#------------------------------------------------------------------------------#",file=f)
    print("# SEQUENCE FOR BULL'S EYE SEQUENCER                                            #",file=f)
    print("# TARGET : {0} | INTLDES {1} / NORAD {2}".format(objname,intldes,norad_id).ljust(79)+"#",file=f)
    print("# Written automatically by cal_satorbit.py (Developed by Kiyoaki Okudaira)     #",file=f)
    print("#------------------------------------------------------------------------------#",file=f)
    print("CONNECTPWI4",file=f)

    if obs_type == "LEO":
        print("GOTORADEC RA={0} DEC={1} AT=ASAP".format(row_elmax["ra[hh:mm:ss.s]"][1:],row_elmax["dec[dd:mm:ss.s]"]),file=f)
        print("SCSOLVEANDSYNC",file=f)
        print("TRACKSAT {0} FROM={1} TO={2}".format(norad_id,seq_begin_lst,seq_end),file=f)
    else:
        print("TRACKSAT {0} FROM=ASAP TO=FALSE".format(norad_id),file=f)
        print("ENABLETRACKSTAR",file=f)
        print("SCSOLVEANDSYNC",file=f)
        print("TRACKSAT {0} FROM=ASAP TO=ENTER".format(norad_id),file=f)
    f.close()

**SharpCap 4.1**<br>
SharpCap 4.1 sequence file (scs file)

In [ ]:
output_filtered = output_filtered[(output_filtered["el[deg]"]>=cap_el_min)]

if obs_ftype == "SER":
    sc_ftype = "SER file (*.ser)"
elif obs_ftype == "FITS":
    sc_ftype = "FITS files (*.fits)"
elif obs_ftype == "PNG":
    sc_ftype = "PNG files (*.png)"
else:
    sc_ftype = "FITS files (*.fits)"

if len(output_filtered) > 0:
    seq_begin = output_filtered[0]["YYYY-MM-DDThh:mm:ss"]
    seq_end   = output_filtered[-1]["YYYY-MM-DDThh:mm:ss"]
    seq_begin_lst = utc2lst(seq_begin,lst_h,lst_m-4,"%H:%M:%S.%f")[:-3]
    
    f = open("{0}/SharpCap/{1}_{2}.scs".format(output_PATH,norad_id,obs_begin[0:10]),mode="w")

    print("#------------------------------------------------------------------------------#",file=f)
    print("# SEQUENCE FOR SharpCap 4.1                                                    #",file=f)
    print("# TARGET : {0} | INTLDES {1} / NORAD {2}".format(objname,intldes,norad_id).ljust(79)+"#",file=f)
    print("# Written automatically by cal_satorbit.py (Developed by Kiyoaki Okudaira)     #",file=f)
    print("#------------------------------------------------------------------------------#",file=f)

    # Camera Settings
    print("SEQUENCE",file=f)
    print("    CAMERA OPEN {0}".format(ccd_name),file=f)
    print("    TARGETNAME {0}".format(norad_id),file=f)
    print("    LIVE MODE",file=f)
    print("    SET COLOUR SPACE TO {0}".format(ccd_colorspace),file=f)
    print("    SET RESOLUTION TO {0}x{1}".format(ccd_resolution_1,ccd_resolution_2),file=f)
    print("    SET BINNING TO {0}".format(ccd_ps_binning),file=f)
    print('    SET OUTPUT FORMAT TO "{0}"'.format(sc_ftype),file=f)
    print("    SET EXPOSURE TO {0}".format(round(ccd_ps_exp,7)),file=f)
    print("    SET GAIN TO {0}".format(ccd_ps_gain),file=f)
    print("    SET OFFSET TO {0}".format(ccd_offset),file=f)
    print("    SET COOLER TARGET TO {0}".format(ccd_cooler),file=f)
    print("    WAIT FOR FRAME",file=f)
    print("    WAIT FOR FRAME",file=f)
    print("    DISPLAY STRETCH AUTO",file=f)
    print("    WAIT UNTIL LATER THAN LOCALTIME {0}".format(seq_begin_lst),file=f)

    # Capture target
    if obs_type == "LEO":
        if obs_timing == "HIGH":
            capture_begin_lst = utc2lst(row_elmax["YYYY-MM-DDThh:mm:ss"],lst_h,lst_m-obsdate_offset,"%H:%M:%S.%f")[:-3]
        else:
            capture_begin_lst = utc2lst(seq_begin,lst_h,lst_m-obsdate_offset,"%H:%M:%S.%f")[:-3]
        if ccd_auto_dur:
            ccd_dur = (datetime.strptime(seq_end, "%Y-%m-%dT%H:%M:%S") - datetime.strptime(seq_begin, "%Y-%m-%dT%H:%M:%S")).seconds
        
        # Mount and Plate solve
        print('    IGNORE ERRORS FROM ONERROR RUN ""',file=f)
        print('        MOUNT GOTO "RA={0}, DEC={1}"'.format(row_elmax["ra[hh:mm:ss.s]"],row_elmax["dec[dd:mm:ss.s]"]),file=f)
        print("        MOUNT SOLVEANDSYNC",file=f)
        print("    END IGNORE ERRORS",file=f)

        # Capture
        print("    SET BINNING TO {0}".format(ccd_binning),file=f)
        print("    SET EXPOSURE TO {0}".format(round(ccd_exp,7)),file=f)
        print("    SET GAIN TO {0}".format(ccd_gain),file=f)
        print("    WAIT FOR FRAME",file=f)
        print("    WAIT FOR FRAME",file=f)
        print("    DISPLAY STRETCH AUTO",file=f)
        print("    WAIT UNTIL LATER THAN LOCALTIME {0}".format(capture_begin_lst),file=f)
        if ccd_capmode == "DUR":
            print("    CAPTURE {0} SECONDS LIVE FRAMES".format(ccd_dur),file=f)
        else:
            print('    CAPTURE {0} FRAMES REQUIREGUIDING False'.format(ccd_frame),file=f)
    else:
        i = 1
        for row in output_filtered:
            capture_begin = row["YYYY-MM-DDThh:mm:ss"]
            capture_begin_lst = utc2lst(capture_begin,lst_h,lst_m-obsdate_offset,"%H:%M:%S.%f")[:-3]

            # Mount and Plate solve
            print("#   Number of captures {0}".format(str(i).rjust(4)),file=f)
            print("    SET BINNING TO {0}".format(ccd_ps_binning),file=f)
            print("    SET EXPOSURE TO {0}".format(ccd_ps_exp),file=f)
            print("    SET GAIN TO {0}".format(ccd_ps_gain),file=f)
            print("    WAIT FOR FRAME",file=f)
            print("    WAIT FOR FRAME",file=f)
            print("    DISPLAY STRETCH AUTO",file=f)
            print('    IGNORE ERRORS FROM ONERROR RUN ""',file=f)
            print('        MOUNT GOTO "RA={0}, DEC={1}"'.format(row["ra[hh:mm:ss.s]"],row["dec[dd:mm:ss.s]"]),file=f)
            print("        MOUNT SOLVEANDSYNC",file=f)
            print("    END IGNORE ERRORS",file=f)

            # Capture
            print("    SET BINNING TO {0}".format(ccd_binning),file=f)
            print("    SET EXPOSURE TO {0}".format(round(ccd_exp,7)),file=f)
            print("    SET GAIN TO {0}".format(ccd_gain),file=f)
            print("    WAIT FOR FRAME",file=f)
            print("    WAIT FOR FRAME",file=f)
            print("    DISPLAY STRETCH AUTO",file=f)
            print("    WAIT UNTIL LATER THAN LOCALTIME {0}".format(capture_begin_lst),file=f)
            if ccd_capmode == "DUR":
                print("    CAPTURE {0} SECONDS LIVE FRAMES".format(ccd_dur),file=f)
            else:
                print('    CAPTURE {0} FRAMES REQUIREGUIDING False'.format(ccd_frame),file=f)
            i = i + 1
    print("    CAMERA CLOSE",file=f)
    print("END SEQUENCE",file=f)
    f.close()